In [ ]:
ptm <- proc.time()

**Import libraries**

In [ ]:
library(rgrass7)
library(rgdal)
library(raster)
library(cluster)
library(randomForest)

**Set GRASS LOCATION** 

    create it if doesn't exist',EPSG:32633

In [ ]:
loc <- initGRASS("/home/epilib/Envs/env1/grass-7.1.svn", home=tempdir())
execGRASS('g.proj',epsg=as.integer(32633), location='utm_wgs84_33N')
execGRASS('g.mapset', mapset='PERMANENT', location='utm_wgs84_33N') 

**Import Bathy**

**Bathy Sub set**

In [ ]:
execGRASS('g.region', raster='elev', res='20', w='580000',e='625000',s='7770000',n='7840000', flags=c('a','p')) 
execGRASS('r.mapcalc', expression = 'minielev=elev', flags='overwrite') 

In [25]:
library(rgrass7)
library(rgdal)
library(raster)

    # Input:
    #    elevation: grid used as input
    #    resolution: (FALSE, Int) manualy set the resolution, FALSE use native dataset resolution
    #    win_neighbors:  int, default:9 - Size in pixel for the neighbors moving window operator.
    #                    (used to compute: average, minimun, maximum elevation)
    #    win_param_scale: int, default:15 - Size in pixel for the param_scale moving window operator.
    #                    (used to compute: profile, cross, minimum, longitudinal curvatures)
    #    curvature_tolerance: float, default: 0.0001 (*)
    #    slope_tolerance: float, default: 0.1 (*)
    #    exponent: float, default: 0.0 (*)
    #    zscale: float, default: 1.0 (*)
    #    Remove: boolean, If TRUE remove intermediate products. default FALSE
    #    
    #    (*) : more info at http://grass.osgeo.org/grass71/manuals/r.param.scale.html
    #
    # Output:
    #    brick raster. Layers: 
    #    'er','xslope','profc','crosc','minic','maxic','longc'

morphoclara <- function(elevation, resolution=FALSE, win_neighbors=9, win_param_scale=15, curvature_tolerance=0.0001, slope_tolerance=0.1, exponent=0.0, zscale=1.0, remove=FALSE ){
if (resolution) {
    execGRASS('g.region',  raster=elevation, res=toString(resolution), flags='p') 
    } else if (!resolution) {
    execGRASS('g.region', raster=elevation, flags='p')
} 
print('average elevation')
execGRASS('r.neighbors', input=elevation, output='avg', size=as.integer(win_neighbors),
          method='average', flags='overwrite')
print('minimum elevation')
execGRASS('r.neighbors', input=elevation, output='min', size=as.integer(win_neighbors),
          method='minimum', flags='overwrite')
print('maximum elevation')
execGRASS('r.neighbors', input=elevation, output='max', size=as.integer(win_neighbors),
          method='maximum', flags='overwrite')
print('normalized elevation')
execGRASS('r.mapcalc', expression = 'er=1.0*(avg-min)/(max-min)', flags='overwrite') 
print('slope')
execGRASS('r.slope.aspect', elevation=elevation, slope='slope', flags='overwrite')
minmax <- read.table(textConnection(execGRASS('r.info', map='slope', flags=c('r'), intern=TRUE)), 
                     header=FALSE, sep="=")
maxslope = minmax[2,2]
expression <- paste('xslope=slope/',maxslope)
execGRASS('r.mapcalc', expression = expression, flags='overwrite') 
print('Profile curvature')
execGRASS('r.param.scale', input=elevation, output='profc', size=as.integer(win_param_scale),
          slope_tolerance=as.numeric(slope_tolerance), curvature_tolerance=as.numeric(curvature_tolerance), 
          method='profc', exponent=as.numeric(exponent), zscale=as.numeric(zscale), flags='overwrite')
print('Cross-sectional curvature')
execGRASS('r.param.scale', input=elevation, output='crosc', size=as.integer(win_param_scale),
          slope_tolerance=as.numeric(slope_tolerance), curvature_tolerance=as.numeric(curvature_tolerance), 
          method='crosc', exponent=as.numeric(exponent), zscale=as.numeric(zscale), flags='overwrite')
print('Minimum curvature')
execGRASS('r.param.scale', input=elevation, output='minic', size=as.integer(win_param_scale),
          slope_tolerance=as.numeric(slope_tolerance), curvature_tolerance=as.numeric(curvature_tolerance), 
          method='minic', exponent=as.numeric(exponent), zscale=as.numeric(zscale), flags='overwrite')
print('Maximum curvature')
execGRASS('r.param.scale', input=elevation, output='maxic', size=as.integer(win_param_scale),
          slope_tolerance=as.numeric(slope_tolerance), curvature_tolerance=as.numeric(curvature_tolerance), 
          method='maxic', exponent=as.numeric(exponent), zscale=as.numeric(zscale), flags='overwrite')
print('Longitudinal curvature')
execGRASS('r.param.scale', input=elevation, output='longc', size=as.integer(win_param_scale),
          slope_tolerance=as.numeric(slope_tolerance), curvature_tolerance=as.numeric(curvature_tolerance), 
          method='longc', exponent=as.numeric(exponent), zscale=as.numeric(zscale), flags='overwrite')                                                                                       
rasters <- readRAST(c('er','xslope','profc','crosc','minic','maxic','longc'), cat=NULL, 
                    ignore.stderr = get.ignore.stderrOption(), NODATA=NULL,
                    mapset=NULL, useGDAL=get.useGDALOption(), close_OK=TRUE, drivername="GTiff",
                    driverFileExt=NULL, return_SGDF=TRUE)
print('Raster stack')
s <- stack(rasters)
names(s) <- c('er','xslope','profc','crosc','minic','maxic','longc')
if (remove) {execGRASS('g.remove', type='raster', name=c('er','xslope','profc','crosc','minic','maxic','longc'), flags='f')
}
return(s)
}


In [ ]:
s <- morphoclara(elevation='minielev', remove=TRUE)

In [ ]:
s

In [ ]:
# we can't operate on the entire set of cells,
# so we use one of the central concepts from statistics: sampling
# sample 10000 random points
s.r <- as.data.frame(sampleRegular(s, 10000))

# clara() function: need to remove NA from training set
s.r <- na.omit(s.r)

ncl <- numeric(8)
for (i in 2:9) ncl[i] <- clara(s.r,stand=TRUE,k=i)$ silinfo $ avg.width
plot(2:10,ncl,type="b")

s.clara <- clara(s.r, stand=TRUE, k=6)
s.r$cluster <- factor(s.clara$clustering)

rf <- randomForest(cluster ~ er + xslope + profc + crosc + minic + maxic
                   + longc, data=s.r, importance=TRUE, ntree=201)

# make predictions from rf model, along all cells of input stack
p <- predict(s, rf, type='response', progress='text')

# variable importance: all contribute about the same... good
#importance(rf)
#varImpPlot(rf)

# customized plot (needs an extra library)
#par(mar=c(0,0,0,0))
#plot(p, maxpixels=50000, axes=FALSE, legend=FALSE, col=brewer.pal('Set1', n=5))

writeRaster(p, "TerClass_sub.asc")

In [ ]:
plot(p)

In [ ]:
execGRASS('r.in.gdal', input='TerClass_sub.asc', output='TerClass_sub', flags=c('overwrite','o','e'))

In [ ]:
execGRASS('r.info', map='TerClass_sub')

In [ ]:
execGRASS('g.region', raster='TerClass_sub', flags=c('a','p')) 

In [ ]:
execGRASS('r.report', flags=c('i','e'), map='TerClass_sub', units=c('me','h','p'), nsteps=255)

In [ ]:
execGRASS('r.univar', map='TerClass_sub')

In [ ]:
tempfile(fileext='.png') 

In [ ]:
proc.time() - ptm